外掛操作

In [ ]:
import torch
print(torch.__version__)

In [ ]:
!pwd #看目前的指定位置
!ls  #看指定位置裡的資料

In [ ]:
from google.colab import drive #掛載到雲端硬碟
drive.mount('/content/gdrive/')

In [ ]:
import os  #位置指定到資料夾
project_path='/content/gdrive/My Drive/Colab Notebooks/hw/hymenoptera_data'
os.chdir(project_path)

使用ImageNet Tiny dataset 做遷移學習 (Transfer Learning)

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
from PIL import Image
from tempfile import TemporaryDirectory

cudnn.benchmark = True
plt.ion()   # interactive mode

影像預處理

In [ ]:
# Data augmentation and normalization for training
# Just normalization for validation
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224), #因應CNN的輸入尺寸皆固定為 224*224
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]) #不可擅自更改, 要不然顏色失真
    ]),
    'val': transforms.Compose([
        transforms.Resize(256), #將圖先校正為256*256
        transforms.CenterCrop(224), #之後再縮小為224*224
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]) #不可擅自更改, 要不然顏色失真
    ]),
}

data_dir = 'hymenoptera_data'
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x])
                  for x in ['train', 'val']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=16, shuffle=True, num_workers=8)
              for x in ['train', 'val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") #為了節省運算效能,故某部分會使用cpu運算

Show Images

In [ ]:
def imshow(inp, title=None):
    """Display image for Tensor."""
    inp = inp.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])  #此處不可任意更改
    std = np.array([0.229, 0.224, 0.225])  #此處不可任意更改
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    plt.imshow(inp)
    if title is not None:
        plt.title(title)
    plt.pause(0.001)  # pause a bit so that plots are updated


# Get a batch of training data
inputs, classes = next(iter(dataloaders['train']))

# Make a grid from batch
out = torchvision.utils.make_grid(inputs)

imshow(out, title=[class_names[x] for x in classes])

***副程式***

訓練模型

In [ ]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    # Create a temporary directory to save training checkpoints
    with TemporaryDirectory() as tempdir:
        best_model_params_path = os.path.join(tempdir, 'best_model_params.pt')

        torch.save(model.state_dict(), best_model_params_path)
        best_acc = 0.0

        for epoch in range(num_epochs):
            print(f'Epoch {epoch}/{num_epochs - 1}')
            print('-' * 10)

            # Each epoch has a training and validation phase
            for phase in ['train', 'val']:
                if phase == 'train':
                    model.train()  # Set model to training mode
                else:
                    model.eval()   # Set model to evaluate mode

                running_loss = 0.0
                running_corrects = 0

                # Iterate over data.
                for inputs, labels in dataloaders[phase]:
                    inputs = inputs.to(device)
                    labels = labels.to(device)

                    # zero the parameter gradients
                    optimizer.zero_grad()

                    # forward
                    # track history if only in train
                    with torch.set_grad_enabled(phase == 'train'):
                        outputs = model(inputs)
                        _, preds = torch.max(outputs, 1)
                        loss = criterion(outputs, labels)

                        # backward + optimize only if in training phase
                        if phase == 'train':
                            loss.backward()
                            optimizer.step()

                    # statistics
                    running_loss += loss.item() * inputs.size(0)
                    running_corrects += torch.sum(preds == labels.data)
                if phase == 'train':
                    scheduler.step()

                epoch_loss = running_loss / dataset_sizes[phase]
                epoch_acc = running_corrects.double() / dataset_sizes[phase]

                print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

                # deep copy the model
                if phase == 'val' and epoch_acc > best_acc:
                    best_acc = epoch_acc
                    torch.save(model.state_dict(), best_model_params_path)

            print()

        time_elapsed = time.time() - since
        print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
        print(f'Best val Acc: {best_acc:4f}')

        # load best model weights
        model.load_state_dict(torch.load(best_model_params_path))
    return model

預測結果的視覺化

In [ ]:
def visualize_model(model, num_images=6):
    was_training = model.training
    model.eval()
    images_so_far = 0
    fig = plt.figure()

    with torch.no_grad():
        for i, (inputs, labels) in enumerate(dataloaders['val']):
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)

            for j in range(inputs.size()[0]):
                images_so_far += 1
                ax = plt.subplot(num_images//2, 2, images_so_far)
                ax.axis('off')
                ax.set_title(f'predicted: {class_names[preds[j]]}')
                imshow(inputs.cpu().data[j])

                if images_so_far == num_images:
                    model.train(mode=was_training)
                    return
        model.train(mode=was_training)

***主程式***



拿ResNet-18做微調 (Fine-Tuing)

In [ ]:
model_conv = torchvision.models.resnet18(weights='IMAGENET1K_V1')
for param in model_conv.parameters():
    param.requires_grad = False

# Parameters of newly constructed modules have requires_grad=True by default
num_ftrs = model_conv.fc.in_features
model_conv.fc = nn.Linear(num_ftrs, 2)

model_conv = model_conv.to(device)

criterion = nn.CrossEntropyLoss()

# Observe that only parameters of final layer are being optimized as
# opposed to before.
optimizer_conv = optim.SGD(model_conv.fc.parameters(), lr=0.001, momentum=0.9) #內部的超參數不能隨意更動

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_conv, step_size=7, gamma=0.1)

訓練模型

In [11]:
model_conv = train_model(model_conv, criterion, optimizer_conv, exp_lr_scheduler, num_epochs=25)

Epoch 0/24
----------
train Loss: 0.6033 Acc: 0.6885
val Loss: 0.4178 Acc: 0.8889

Epoch 1/24
----------
train Loss: 0.4105 Acc: 0.8115
val Loss: 0.2967 Acc: 0.8954

Epoch 2/24
----------
train Loss: 0.3797 Acc: 0.8402
val Loss: 0.2129 Acc: 0.9542

Epoch 3/24
----------
train Loss: 0.2563 Acc: 0.9057
val Loss: 0.1979 Acc: 0.9477

Epoch 4/24
----------
train Loss: 0.2432 Acc: 0.9098
val Loss: 0.1881 Acc: 0.9542

Epoch 5/24
----------
train Loss: 0.2788 Acc: 0.9016
val Loss: 0.1952 Acc: 0.9346

Epoch 6/24
----------
train Loss: 0.2315 Acc: 0.9221
val Loss: 0.2046 Acc: 0.9150

Epoch 7/24
----------
train Loss: 0.2934 Acc: 0.8648
val Loss: 0.1631 Acc: 0.9542

Epoch 8/24
----------
train Loss: 0.2140 Acc: 0.9262
val Loss: 0.1477 Acc: 0.9673

Epoch 9/24
----------
train Loss: 0.1890 Acc: 0.9262
val Loss: 0.1622 Acc: 0.9542

Epoch 10/24
----------
train Loss: 0.1943 Acc: 0.9385
val Loss: 0.1536 Acc: 0.9673

Epoch 11/24
----------
train Loss: 0.1952 Acc: 0.9221
val Loss: 0.1517 Acc: 0.9608

Ep

KeyboardInterrupt: ignored

預測影像的視覺化

In [ ]:
visualize_model(model_conv)

plt.ioff()
plt.show()